In [30]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

In [31]:
import h5py
def load_data():
    f = h5py.File('../hdf5/shapes_classification.hdf5', 'r')
    return (f['train_img'][...],f['train_labels'][...]),(f['test_img'][...],f['test_labels'][...])

In [32]:
(train_images,train_labels),(test_images,test_labels) = load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0
print(train_images.shape, type(train_images))
print(train_labels.shape, type(train_labels))

(200, 64, 64, 1) <class 'numpy.ndarray'>
(200,) <class 'numpy.ndarray'>


In [33]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(train_images.shape[1],
                                     train_images.shape[2],
                                     train_images.shape[3])))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

In [34]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 29, 29, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 12, 12, 64)        36928     
_________________________________________________________________
flatten_2 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)               

In [35]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10,
                    validation_data=(test_images, test_labels))


Epoch 1/10
7/7 [==============================] - 1s 125ms/step - loss: 1.6106 - accuracy: 0.2000 - val_loss: 1.6159 - val_accuracy: 0.2000
Epoch 2/10
7/7 [==============================] - 1s 92ms/step - loss: 1.6078 - accuracy: 0.2100 - val_loss: 1.6144 - val_accuracy: 0.1800